In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
import emoji
import string
from nltk import sent_tokenize
from nltk import word_tokenize
#from keras.preprocessing.sequence import pad_sequences


/home/minhto/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# READ DATA
train_data = pd.read_csv('Data/train.txt', sep="### ", header=None, engine='python')
drop_list = train_data.index[train_data[0] == 4].tolist() # list of data with label 4
train_data = train_data.drop(train_data.index[drop_list]) # drop these rows
train_data = train_data.reset_index(drop=True)


In [4]:
train_data[1][1]

'ad ơi  . mình ở bình định  . mình đang sử dụng Internet của VNPT  . gói cước 132000/tháng . ad có thể cho mình biết thông tin về tốc độ đường truyền của gói này được k ad'

In [5]:
# EACH COMMENT TO SENTENCES
all_sentences = []
for i in range(len(train_data)):
    tokenized = sent_tokenize(train_data[1][i])
    for element in range(len(tokenized)):
        all_sentences.append(tokenized[element])



In [6]:
# characters that needs to be removed
other_removal = ['0','1','2','3','4','5','6','7','8','9','«','“','”','…','»','】☛','►►','🏿','➤','◄◄','🏻','•','·','►']

punctuation = list(string.punctuation)

removal_character = other_removal + list(emoji.UNICODE_EMOJI.keys()) + punctuation


In [7]:
def remove_character(sentence):
    sentence_split = ""
    for word in sentence:
        for ch in word:
            word = ''.join(ch for ch in word if ch not in removal_character)
            sentence_split += word
    return sentence_split


In [8]:
def sentence2word(sentence):
    tokenized = word_tokenize(sentence)
    return tokenized



In [54]:
# CREATE BAG OF WORDS
boW = []
clean_sentences = []

for i in range(len(all_sentences)):
    clean_sentence = remove_character(all_sentences[i])
    tokenized = word_tokenize(clean_sentence)
    clean_sentences.append(tokenized)   # all clean sentences
    for word in tokenized:
        boW.append(word.lower())  # BoW


In [57]:
result = False 
for sentence in clean_sentences:
    if 'ngy' in sentence:
        result = True
print(result)

False


In [116]:
# remove too long words and stop words
long_words = []
for ch in boW:
    if len(ch) > 14:
        long_words.append(ch)

vnese_stop_words = ['bị','bởi','cả','các','cái','cần','càng','chỉ','chiếc','cho','chứ','chưa','chuyện','có','có_thể','cứ',
                    'của','cùng','cũng','đã','đang','đây','để','đến_nỗi','đều','điều','do','đó','được','dưới','gì',
                    'khi','không','là','lại','lên','lúc','mà','mỗi','một_cách','này','nên','nếu','ngay','nhiều','như',
                    'nhưng','những','nơi','nữa','phải','qua','ra','rằng','rất','rồi','sau','sẽ','so','sự','tại','theo',
                    'thì','trên','trước','từ','từng','và','vẫn','vào','vậy','vì','việc','với','vừa']

boW = [word for word in boW if word not in vnese_stop_words not in long_words]
boW = list(set(boW))
print('number of words in bag of words now is: %d' % len(boW))

number of words in bag of words now is: 9067


In [55]:
'ngy' in boW

False

In [40]:
word2int = {}
int2word = {}


for i, word in enumerate(boW_):
    word2int[word] = i
    int2word[i] = word

word2int.update({'0': 0})  # for padding sentences with 0
int2word.update({'0': '0'})  # for padding sentences with 0


In [58]:
# MODIFY AND PADDING CLEAN SENTENCES
for sentence in clean_sentences:
    for word in sentence:
        if len(word) > 14:
            sentence.remove(word)


In [59]:
result = False
for sentence in clean_sentences:
    if 'ngy' in sentence:
        result = True
print(result)

False


In [60]:
def check_stopwords(sentence):
    for word in sentence:
        if word in vnese_stop_words:
            return True 
    return False

In [61]:
clean_sw_sentences = []
for i in range(len(clean_sentences)):
    rm_stopword = []
    rm_stopword += [word for word in clean_sentences[7] if word not in vnese_stop_words]
    clean_sw_sentences.extend(rm_stopword)

In [62]:
def check_stopwords(sentence):
    for word in sentence:
        if word in vnese_stop_words:
            return True 
    return False

count = 0

for i in range(len(clean_sentences)):
    if check_stopwords(clean_sentences[i]):
        count += 1

print(count)

8892


In [64]:
for i in range(len(clean_sentences)):
    clean_sentences[i] = [word.lower() if any(x.isupper() for x in word) else word for word in clean_sentences[i]]
                

In [65]:
clean_sentences[1]
def check_upper(sentence):
    for word in sentence:
        for x in word: 
            if x.isupper():
                return True
    return False

count = 0
for i in range(len(clean_sentences)):
    if check_upper(clean_sentences[i]):
        count += 1
        
print(count)

0


In [47]:
def longest_sentence(sentences):
    position = 0
    longest = 0
    for num, sentence in enumerate(sentences):
        if len(sentences[num]) > longest:
            longest = len(sentences[num])
            position = num

    return position, longest


In [48]:
def longest_word(sentences):
    longest_word_length = 0
    long_word = ""
    for sentence in sentences:
        for word in sentence:
            if len(word) > longest_word_length:
                longest_word_length = len(word)
                long_word = word
    return long_word


In [49]:
print('the longest word is %s' %(longest_word(clean_sentences)))
pos, longest_num = longest_sentence(clean_sentences)
print('pos is %d, longest_num is %d' % longest_sentence(clean_sentences))
#print(clean_sentences[2135])

the longest word is httpvnreviewvn
pos is 2135, longest_num is 169


In [50]:
def pad(sentences, longest):
    for i in range(len(sentences)):
        if len(sentences[i]) < longest:
            for j in range(longest - len(sentences[i])):
                sentences[i] += '0'


#pad(clean_sentences, longest_num)


In [66]:
data = []

window_size = 2

for sentence in clean_sentences: 
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - window_size, 0) : min(word_index + window_size, len(sentence)) + 1]:
            if nb_word != word: 
                data.append([word, nb_word])
data

[['em', 'mua'],
 ['em', 'đc'],
 ['mua', 'em'],
 ['mua', 'đc'],
 ['mua', 'ngày'],
 ['đc', 'em'],
 ['đc', 'mua'],
 ['đc', 'ngày'],
 ['đc', 'hỏng'],
 ['ngày', 'mua'],
 ['ngày', 'đc'],
 ['ngày', 'hỏng'],
 ['ngày', 'shop'],
 ['hỏng', 'đc'],
 ['hỏng', 'ngày'],
 ['hỏng', 'shop'],
 ['hỏng', 'ko'],
 ['shop', 'ngày'],
 ['shop', 'hỏng'],
 ['shop', 'ko'],
 ['shop', 'thèm'],
 ['ko', 'hỏng'],
 ['ko', 'shop'],
 ['ko', 'thèm'],
 ['ko', 'nói'],
 ['thèm', 'shop'],
 ['thèm', 'ko'],
 ['thèm', 'nói'],
 ['thèm', 'gì'],
 ['nói', 'ko'],
 ['nói', 'thèm'],
 ['nói', 'gì'],
 ['nói', 'luôn'],
 ['gì', 'thèm'],
 ['gì', 'nói'],
 ['gì', 'luôn'],
 ['gì', 'nói'],
 ['luôn', 'nói'],
 ['luôn', 'gì'],
 ['luôn', 'nói'],
 ['luôn', 'mỗi'],
 ['nói', 'gì'],
 ['nói', 'luôn'],
 ['nói', 'mỗi'],
 ['nói', 'câu'],
 ['mỗi', 'luôn'],
 ['mỗi', 'nói'],
 ['mỗi', 'câu'],
 ['mỗi', 'lỗi'],
 ['câu', 'nói'],
 ['câu', 'mỗi'],
 ['câu', 'lỗi'],
 ['câu', 'do'],
 ['lỗi', 'mỗi'],
 ['lỗi', 'câu'],
 ['lỗi', 'do'],
 ['lỗi', 'người'],
 ['do', 'câu'],
 ['

In [130]:
'\202akotex'.encode('ascii','ignore')

b'akotex'

In [133]:
data[3][1]

'đc'

In [135]:
data[3][1].encode('ascii', 'ignore').decode("utf-8")

'c'

In [131]:
data1 = []
data2 = []
for i in range(len(data)):
    data1.append(data_word[0].encode('ascii', 'ignore'))
    data2.append(data_word[1].encode('ascii', 'ignore'))
    
data1 = list(set(data1))
data2 = list(set(data2))

print(len(data1))
print(len(data2))

1
1


In [107]:
'ngy' in data1

False

In [88]:
result = False
for data_element in data: 
    if 'ngy' in data_element[0] or 'ngy' in data_element[1]:
        result = True
print(result)

True


In [100]:
boW2 = []
for data_element in data: 
    boW2.extend(data_element)
boW2 = list(set(boW2))
len(boW2)

8747

In [101]:
'ngy' in boW2

False

In [115]:
result = False
for data_word in data: 
    if 'ngy' in data_word[0] or 'ngy' in data_word[1]:
        result = True
print(result)    

True


In [114]:
result = 7
for data_word in data: 
    if 'ngy' in data_word[0]:
        result = 1 
    else:
        result = 0
print(result)

0


In [ ]:
vocab_size = len(boW_)
def to_one_hot(index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[index] = 1
    return temp

x_train = []
y_train = []

for data_word in data: 
    x_train.append(to_one_hot(word2int[data_word[0]], vocab_size))
    y_train.append(to_one_hot(word2int[data_word[1]], vocab_size))
    
x_train.head()

In [ ]:
from __future__ import print_function
import torch